In [ ]:
import numpy as np
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
import random
import time
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
import undetected_chromedriver as uc
import random

options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-notifications")

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
]
options.add_argument(f"user-agent={random.choice(user_agents)}")

# Use version_main matching your Chrome version
browser = uc.Chrome(options=options, version_main=139)

# Hide webdriver
browser.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")


In [ ]:
def slow_type(element, text, delay_range=(0.05, 0.15)):
    for char in text:
        element.send_keys(char)
        time.sleep(random.uniform(*delay_range))

def human_scrolls(browser, max_scrolls=10):
    last_height = browser.execute_script("return document.body.scrollHeight")

    for i in range(max_scrolls):
        scroll_distance = random.randint(300, 800)

        browser.execute_script(f"window.scrollBy(0, {scroll_distance});")

        time.sleep(random.uniform(0.8, 2.5))

        if random.random() < 0.2:
            time.sleep(random.uniform(3, 5))

        new_height = browser.execute_script("return window.scrollY + window.innerHeight")
        total_height = browser.execute_script("return document.body.scrollHeight")

        if new_height >= total_height - 50:
            break


def popup_close():
    try:
        popup_close_btn = WebDriverWait(browser, 2).until(
            EC.element_to_be_clickable((
                By.CSS_SELECTOR, "button[data-test='job-alert-modal-close']"
            ))
        )
        popup_close_btn.click()
        print("Pop Up Closed")
    except:
        pass


def human_scroll(scroll_increment_range=(200, 400), pause_range=(0.5, 1.5), long_pause_chance=0.1):
    last_scroll = -1
    while True:
        scroll_height = browser.execute_script("return document.body.scrollHeight")
        current_scroll = browser.execute_script("return window.pageYOffset")

        if current_scroll == last_scroll:
            print("Reached end of page, stopping scroll")
            break
        last_scroll = current_scroll

        scroll_step = random.randint(*scroll_increment_range)
        browser.execute_script(f"window.scrollBy(0, {scroll_step});")
        time.sleep(random.uniform(*pause_range))

        if random.random() < long_pause_chance:
            time.sleep(random.uniform(2, 5))


def human_scroll_to_top(scroll_increment_range=(200, 400), pause_range=(0.5, 1.5), long_pause_chance=0.1):
    current_position = browser.execute_script("return window.pageYOffset")
    
    while current_position > 0:
        scroll_step = random.randint(*scroll_increment_range)
        current_position -= scroll_step
        if current_position < 0:
            current_position = 0
        browser.execute_script(f"window.scrollTo(0, {current_position});")
        time.sleep(random.uniform(*pause_range))
        
        if random.random() < long_pause_chance:
            time.sleep(random.uniform(1, 3))
        
        if random.random() < 0.05:
            scroll_down = random.randint(50, 150)
            current_position += scroll_down
            total_height = browser.execute_script("return document.body.scrollHeight")
            if current_position > total_height:
                current_position = total_height
            browser.execute_script(f"window.scrollTo(0, {current_position});")
            time.sleep(random.uniform(0.2, 0.5))



In [ ]:

try:
    url = """https://www.glassdoor.co.in/index.htm"""
    browser.get(url)

    if random.random() < 0.7:
        time.sleep(random.uniform(6, 8))
    else:
        time.sleep(random.uniform(10, 15))

    my_mail = "xy@xy.com"
    password = 'xy'

    wait = WebDriverWait(browser, 10)

    time.sleep(random.uniform(1.5, 4))

    input_box = wait.until(EC.visibility_of_element_located((
        By.ID, "inlineUserEmail"
    )))
    time.sleep(random.uniform(0.2, 0.5))
    input_box.clear()
    slow_type(input_box, my_mail)
    print("✅ Phone/email entered.")

    time.sleep(random.uniform(1.5, 5))

    next_button = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR, "button.Button[type='submit'][data-test='continue-with-email-inline']"
    )))
    next_button.click()

    password_box = wait.until(EC.visibility_of_element_located((
        By.ID, "inlineUserPassword"
    )))
    time.sleep(random.uniform(0.2, 0.5))
    password_box.clear()
    slow_type(password_box, password)
    print("✅ Password entered.")

    time.sleep(random.uniform(4, 6))

    sign_in_button = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR, "button.Button[type='submit']"
    )))
    sign_in_button.click()
    print("✅ Logined")

except TimeoutException as e:
    print("⏳ Timeout waiting for element:", e)
except NoSuchElementException as e:
    print("❌ Element not found:", e)
except Exception as e:
    print("⚠️ Unexpected error:", e)

In [ ]:
if random.random() < 0.7:
    time.sleep(random.uniform(5, 8))
else:
    time.sleep(random.uniform(8, 12))

human_scrolls(browser, max_scrolls=6)


job_title = input("Enter job title (e.g. Data Analyst, Java Developer): ").strip()
days = int(input("Enter time filter in days (0 = All jobs, 1 = Last 24h, 3 = Last 3 days, 7 = Last 7 days): ").strip())

job_title_slug = job_title.lower().replace(" ", "-")

location_slug = "india"

base_url = f"https://www.glassdoor.co.in/Job/{location_slug}-{job_title_slug}-jobs-SRCH_IL.0,5_IN115_KO6,18.htm"

if days > 0:
    job_url = f"{base_url}?fromAge={days}"
else:
    job_url = base_url

print("Generated Glassdoor Job URL:", job_url)

# https://www.glassdoor.co.in/Job/india-data-analyst-jobs-SRCH_IL.0,5_IN115_KO6,18.htm?fromAge=1
browser.get(job_url)


if random.random() < 0.7:
    time.sleep(random.uniform(3, 6))
else:
    time.sleep(random.uniform(6, 10))

popup_close()

In [ ]:
popup_close()
popup_close()

while True:
    human_scroll()
    popup_close()

    try:
        show_more = wait.until(EC.element_to_be_clickable((
            By.CSS_SELECTOR, "button[data-test='load-more']"
        )))
        browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", show_more)
        time.sleep(random.uniform(0.5, 1.5))
        show_more.click()
        print("Clicked Show More")
        time.sleep(random.uniform(2, 4))
        popup_close()
    except:
        print("No more Show More button, all jobs loaded")
        break
        
human_scroll_to_top()

job_cards = wait.until(EC.presence_of_all_elements_located(
    (By.CSS_SELECTOR, "li[data-test='jobListing']")
))

print('Total Jobs', len(job_cards))

all_jobs = []
link_set = set()

for idx, job_card in enumerate(job_cards):

    job_title = job_link = company_name = location = job_description = None
    
    try:
        browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", job_card)
        time.sleep(random.uniform(1, 2.5))

        job_card.click()
        time.sleep(random.uniform(2, 4))
        popup_close()

        # ------------------------
        # Job description
        # ------------------------
        try:
            try:
                expand = wait.until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-test='show-more-cta']"))
                )
                browser.execute_script("arguments[0].click();", expand)
                print("Expanded job description")
                time.sleep(1.5)
            except:
                print("ℹ️ No expand button for this job")

            try:
                scrollable_div = wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.TwoColumnLayout_jobDetailsContainer__qyvJZ"))
                )
                last_height = 0
                while True:
                    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)
                    time.sleep(random.uniform(1.5, 3.0))
                    new_height = browser.execute_script("return arguments[0].scrollHeight;", scrollable_div)
                    if new_height == last_height:
                        break
                    last_height = new_height
            except Exception as e:
                print(f"⚠️ Could not scroll job description: {e}")

            job_description = wait.until(
                EC.presence_of_element_located((
                    By.CSS_SELECTOR, "section.Section_sectionComponent__nRsB2.Section_bottomMargin__Fka0J"
                ))
            ).text.strip()

        except Exception as e:
            print(f"⚠️ Job {idx} JD slow, retrying via neighbor: {e}")
            job_description = None
            try:
                if idx > 0:
                    neighbor = job_cards[idx-1]
                else:
                    neighbor = job_cards[idx+1]

                browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", neighbor)
                time.sleep(random.uniform(1, 2))
                neighbor.click()
                time.sleep(random.uniform(1.5, 3))

                browser.execute_script("arguments[0].scrollIntoView({block: 'center'});", job_card)
                time.sleep(random.uniform(1, 2))
                job_card.click()
                time.sleep(random.uniform(2, 4))
                popup_close()

                try:
                    expand = browser.find_element(By.CSS_SELECTOR, "button[data-test='show-more-cta']")
                    if expand.is_displayed():
                        browser.execute_script("arguments[0].click();", expand)
                        time.sleep(1.5)
                except:
                    pass

                scrollable_div = browser.find_element(By.CSS_SELECTOR, "div.TwoColumnLayout_jobDetailsContainer__qyvJZ")
                browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_div)
                time.sleep(2)

                job_description = wait.until(
                    EC.presence_of_element_located((
                        By.CSS_SELECTOR, "section.Section_sectionComponent__nRsB2.Section_bottomMargin__Fka0J"
                    ))
                ).text.strip()

            except Exception as e2:
                print(f"❌ Still could not extract JD after retry: {e2}")
                job_description = None

        # ------------------------
        # Job title
        # ------------------------
        try:
            job_title = wait.until(EC.visibility_of_element_located((
                By.CSS_SELECTOR, "h1.heading_Heading__aomVx.heading_Level1__w42c9"
            ))).text.strip()
        except Exception as e:
            print(f"⚠️ Could not extract job title: {e}")
            job_title = None

        # ------------------------
        # Job link
        # ------------------------
        try:
            job_links = browser.find_elements(
                By.CSS_SELECTOR, "a[data-test='job-title']"
            )
            job_link = job_links[idx].get_attribute("href").strip()
        
            if job_link in link_set:
                print("Already Scraped:", job_link)
                continue
            link_set.add(job_link)
        except Exception as e:
            print(f"⚠️ Could not extract job link for job {idx}: {e}")
            job_link = None


        # ------------------------
        # Company
        # ------------------------
        try:
            company_name = browser.find_element(
                By.CSS_SELECTOR, "div.EmployerProfile_employerNameHeading__bXBYr h4"
            ).text.strip()
        except Exception as e:
            print(f"⚠️ Could not extract company name: {e}")
            company_name = None

        # ------------------------
        # Location
        # ------------------------
        try:
            location = browser.find_element(
                By.CSS_SELECTOR, "div[data-test='location']"
            ).text.strip()
        except Exception as e:
            print(f"⚠️ Could not extract location: {e}")
            location = None

        # print(job_description)
        # print("*"*100)
        all_jobs.append([job_title, job_link, company_name, location, job_description])
        print("*"*100)
        print(f"{idx} | Title: {job_title} | Company: {company_name} | Location: {location} | "
              f"Link length: {len(job_link) if job_link else 0} | "
              f"JD length: {len(job_description) if job_description else 0}")
        print("*"*100)

    except Exception as e:
        print(f"❌ Error on job {idx}: {e}")
        continue


In [ ]:
df = pd.DataFrame(all_jobs, columns=["Job Title", "Job Link", "Company", "Location", "Job Desc."])
df.head()

In [ ]:
# print(df['Job Desc.'][0])

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.to_csv('glassdoor_jobs.csv', index=False)

In [ ]:
def sign_out(max_retries=4):
    for attempt in range(1, max_retries + 1):
        try:
            browser.execute_script("window.scrollBy(0, -200);")
            time.sleep(random.uniform(2, 4))
            profile_button = wait.until(EC.element_to_be_clickable((
                By.CSS_SELECTOR, "button[data-test='utility-nav-profile-button'], button[data-test='mobile-utility-nav-profile-button']"
            )))
            profile_button.click()
            time.sleep(random.uniform(1, 2))

            sign_out_link = wait.until(EC.element_to_be_clickable((
                By.LINK_TEXT, "Sign out"
            )))
            sign_out_link.click()

            print(f"✅ Signed out successfully on attempt {attempt}.")
            return  # exit once successful

        except Exception as e:
            print(f"⚠️ Attempt {attempt} failed: {e}")
            if attempt < max_retries:
                time.sleep(random.uniform(2, 4))  # wait before retry
            else:
                print("❌ Could not sign out after 4 attempts.")

# Call it
sign_out()
